In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import time
from PIL import Image
import json
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
batch_size = 64
image_size = 224

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
validation_generator = validation_datagen.flow_from_directory(
    valid_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
num_classes = len(train_generator.class_indices)
class_names = list(train_generator.class_indices.keys())

In [ ]:
print(f"Number of flower categories: {num_classes}")
print(f"Category names: {class_names}")

In [ ]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL, input_shape=(image_size, image_size, 3), trainable=False)

In [ ]:
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=0.00001
)

In [ ]:
epochs = 20
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")

In [ ]:
def plot_training_curves(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs_range = range(len(acc))
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    
    plt.savefig('training_curves.png')
    plt.show()

In [ ]:
plot_training_curves(history)

In [ ]:
model.save('my_model.h5')
print("Model saved successfully!")

In [ ]:
class_indices = train_generator.class_indices
label_map = {str(v): k for k, v in class_indices.items()}

In [ ]:
with open('label_map.json', 'w') as f:
    json.dump(label_map, f)

In [ ]:
print("Label map (label_map.json) created successfully!")

In [ ]:
def process_image(image_path):
    image = Image.open(image_path)
    image = image.resize((image_size, image_size))
    image = np.array(image) / 255.0
    return image

In [ ]:
def predict(image_path, model, top_k=5):
    image = process_image(image_path)
    image = np.expand_dims(image, axis=0)
    
    predictions = model.predict(image)
    top_values, top_indices = tf.nn.top_k(predictions, k=top_k)
    
    top_classes = [label_map[str(idx)] for idx in top_indices.numpy()[0]]
    top_probs = top_values.numpy()[0]
    
    return top_probs, top_classes

In [ ]:
test_images = [os.path.join(test_dir, os.listdir(test_dir)[0], os.listdir(os.path.join(test_dir, os.listdir(test_dir)[0]))[0])]

In [ ]:
for image_path in test_images:
    probs, classes = predict(image_path, model)
    
    plt.figure(figsize=(6, 10))
    
    plt.subplot(2, 1, 1)
    img = plt.imread(image_path)
    plt.imshow(img)
    plt.title(f'Image: {os.path.basename(image_path)}')
    plt.axis('off')
    
    plt.subplot(2, 1, 2)
    plt.barh(range(len(probs)), probs)
    plt.yticks(range(len(probs)), classes)
    plt.xlabel('Probability')
    plt.title('Top Predictions')
    
    plt.tight_layout()
    plt.savefig('prediction_example.png')
    plt.show()

In [ ]:
print("Image classifier model development, training, and evaluation completed!")